In [1]:
import sys
sys.path.append("./CVAE")

In [2]:
!pip install pyro-ppl
from main import main
import argparse
import itertools
import pandas as pd
import shutil
import os
import glob

In [4]:
# Define the hyperparameter search space
learning_rates = [1e-4, 1e-3]
num_epochs_list = [50, 101]
num_particles_list = [20, 10]
z_dims = [200, 400]
random_mask = True

# Iterate over all combinations of hyperparameters
for lr, num_epochs, num_particles, z_dim in itertools.product(learning_rates, num_epochs_list, num_particles_list, z_dims):
    # Set up arguments
    args = argparse.Namespace(
        num_quadrant_inputs=[1, 2, 3],
        num_epochs=num_epochs,
        early_stop_patience=3,
        learning_rate=lr,
        cuda=True,
        num_images=10,
        num_samples=10,
        num_particles=num_particles,
        dataset='cifar10',
        z_dim=z_dim,
        random_mask=random_mask,
        hidden_1=500,
        hidden_2=500,
        allow_baseline_reuse=False,
        use_conv=False
    )

    # Log the current configuration
    print(f"Running with lr={lr}, num_epochs={num_epochs}, num_particles={num_particles}, z-dim={z_dim}")

    # Run the main function
    try:
        main(args)
    except Exception as e:
        print(f"Error with configuration lr={lr}, num_epochs={num_epochs}, num_particles={num_particles}, z-dim={z_dim}: {e}")
        continue

    # Save the results
    if os.path.exists("results.csv"):
        output_file = f"tuning_results/results_lr_{lr}_epochs_{num_epochs}_particles_{num_particles}_z_{z_dim}.csv"
        shutil.copy("results.csv", output_file)
        print(f"Results saved to {output_file}")
    else:
        print(f"No 'results.csv' file generated for lr={lr}, num_epochs={num_epochs}, num_particles={num_particles}_z_{z_dim}.")
    # Backup important files before starting

    for q in range(1, 3):
        file_name = "cvae{}_plot_q{}.png".format("_rand_mask" if random_mask else "", q)
        if os.path.exists(file_name):
            output_file = f"tuning_results/cvae_plot{'_rand_mask' if random_mask else ''}_q{q}_lr_{lr}_epochs_{num_epochs}_particles_{num_particles}_z_{z_dim}.png"
            shutil.copy(file_name, output_file)
            print(f"Backup created for '{file_name}'.")


Running with lr=0.0001, num_epochs=1, num_particles=20, z-dim=200
Device:  cuda:0
Using random mask
Training with 1 quadrant as input...
Files already downloaded and verified
Files already downloaded and verified


CVAE Epoch 0 val    : 100%|██████████| 79/79 [00:02<00:00, 37.60it/s, early_stop_count=0, loss=1485.54]


Files already downloaded and verified
Files already downloaded and verified


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [-1.0..1.0].


Files already downloaded and verified
Files already downloaded and verified


Generating predictions: 100%|██████████| 313/313 [00:22<00:00, 13.75it/s]


Df:                       1 quadrant
NN (baseline)       1964.947456
CVAE (Monte Carlo)   1511.83589
Columns:  ['1 quadrant']
Training with 2 quadrants as input...
Files already downloaded and verified
Files already downloaded and verified


NN Epoch 0 train    :  51%|█████     | 198/391 [00:05<00:05, 34.27it/s, early_stop_count=0, loss=1014.35]


KeyboardInterrupt: 

<Figure size 640x480 with 0 Axes>

In [ ]:
# Combine all results for analysis
result_files = glob.glob("tuning_results/*.csv")
if result_files:
    combined_results = []

    for file in result_files:
        # Read each result file
        df = pd.read_csv(file)

        # Rename the first column if necessary
        if 'Unnamed: 0' in df.columns:
            df.rename(columns={'Unnamed: 0': 'Method'}, inplace=True)

        print(df)

        # Extract performance gaps and compute the aggregate gap
        performance_gap_row = df.loc[df['Method'] == 'Performance gap']
        aggregate_gap = performance_gap_row[['1 quadrant', '2 quadrants', '3 quadrants']].sum(axis=1).values[0]

        # Store results with the file name and aggregate gap
        combined_results.append({
            "file": file,
            "aggregate_gap": aggregate_gap
        })

    # Convert to a DataFrame for sorting and analysis
    combined_results_df = pd.DataFrame(combined_results)

    # Sort by aggregate gap in descending order (largest gap first)
    combined_results_df = combined_results_df.sort_values(by="aggregate_gap", ascending=False)

    # Save the combined results to a file
    combined_results_df.to_csv("tuning_results/combined_results.csv", index=False)
    print("All results combined and saved to 'tuning_results/combined_results.csv'.")

    # Display the best configuration
    print("Best configuration:")
    best_file = combined_results_df.iloc[0]['file']
    print(f"File: {best_file}, Aggregate Performance Gap: {combined_results_df.iloc[0]['aggregate_gap']}")

    # Optionally, load and display the details of the best result
    best_result = pd.read_csv(best_file)
    print("\nDetails of the best configuration:")
    print(best_result)
else:
    print("No results to combine.")


In [ ]:
# zip all the files in tuning_results folder
import tarfile
import os
import glob

with tarfile.open("tuning_results.tar.gz", "w:gz") as tar:
    print("num of files:", len(glob.glob("tuning_results/*")))
    for file in glob.glob("./tuning_results/*"):
        tar.add(file, arcname=os.path.basename(file))
